In [8]:
def process_report(csv_file_path):
    import pandas as pd
    import re
    from modules import get_current_date_formated, get_cursor
    from app import execute_query, printError, printLog
    
    cur, connection = get_cursor()
    
    try:
        # Load the CSV file into a DataFrame
        df = pd.read_csv(csv_file_path)
        print("--Report downloaded. Extracting content...")
        df.fillna(0, inplace=True)
        print(f"--{len(df)} records available.")
        
        cleaned_columns = ['RID', 'Order Date', 'Order No', 'Order Status', 'Order Category', 'Cancelled By?',
                           "Item's total", 'Packing & Service charges', 'Merchant Discount', 
                           'Net Bill Value (without taxes)', 'GST liability of  Merchant', 
                           'Customer payable (Net bill value after taxes & discount)', 
                           'Swiggy Platform Service Fee Chargeable On', 'Swiggy Platform Service Fee %', 
                           'Discount on Swiggy Platform Service Fee', 'Total Long Distance Subscription Fees', 
                           'Total Discount on Long Distance Subscription Fees', 
                           'Total Effective Long Distance Subscription Fees', 'Collection Charges', 
                           'Access Charges', 'Merchant Cancellation Charges', 'Call Center Service Fees', 
                           'Total Swiggy Service fee (without taxes)', 'Delivery fee (sponsored by merchant)', 
                           'Taxes on Swiggy fee', 'Total Swiggy fee (including taxes)', 
                           'Cash Prepayment to Merchant', 'Merchant Share of Cancelled Orders', 
                           'GST Deduction U/S 9(5)', 'Refund for Disputed Order', 'Disputed Order Remarks', 
                           'Total of Order Level Adjustments', 'Net Payable Amount (before TCS deduction)', 
                           'Net Payable Amount (after TCS and TDS deduction)', 'MFR Pressed?', 
                           'Cancellation Policy Applied', 'Coupon Code Sourced', 'Discount Campaign ID', 
                           'Is_replicated', 'Base order ID', 'MRP Items', 'Order Payment Type', 
                           'Cancellation time', 'Pick Up Status', 'Coupon code applied by customer', 
                           'Nodal UTR', 'Current UTR', 'Long Distance Applicable', 'Last Mile Distance']
        
        # Rename columns if necessary
        for col in cleaned_columns:
            for df_col in df.columns:
                if col in df_col:
                    df.rename(columns={df_col: col}, inplace=True)
                    break
        
        # Ensure 'Order No' is numeric
        df['Order No'] = df['Order No'].apply(lambda x: re.sub(r'\D', '', str(x)).strip())
        df['Order No'] = pd.to_numeric(df['Order No'], errors='coerce')  # Convert to numeric, coercing errors to NaN
        
        # Drop rows where 'Order No' is NaN (couldn't convert to number)
        df.dropna(subset=['Order No'], inplace=True)
        
    except Exception as e:
        e_type = type(e).__name__
        if e_type == 'EmptyDataError':
            printError(e, False)
        printLog("--No data available for this date.")
        return False
    
    data = []
    
    for i in range(len(df)):
        record = df.iloc[i]
        
        order_no = str(int(record['Order No'])).strip()  # Ensure 'Order No' is a string and integer
        restaurant_id = str(record['RID']).strip()
        
        dct = {
            "order_id": order_no,
            "restaurant_id": restaurant_id,
            "order_date": record['Order Date'],
            "created_at": get_current_date_formated(),
            "updated_at": get_current_date_formated(),
            "order_status": record['Order Status'],
            "order_category": record['Order Category'],
            "cancelled_by": record["Cancelled By?"],
            "item_total": record["Item's total"],
            "packing_and_service_charges": record["Packing & Service charges"],
            "merchant_discount": record["Merchant Discount"],
            "net_amount_without_taxes": record["Net Bill Value (without taxes)"],
            "gst_liability_of_merchant": record["GST liability of  Merchant"],
            "customer_payable_net_after_tax_and_discount": record["Customer payable (Net bill value after taxes & discount)"],
            "platform_fee_chargeable_on": record["Swiggy Platform Service Fee Chargeable On"],
            "platform_fee_percent": record["Swiggy Platform Service Fee %"],
            "platform_fee": record["Swiggy Platform Service Fee G"],
            "discount_on_platform_fee": record["Discount on Swiggy Platform Service Fee"],
            "collection_charges": record["Collection Charges"],
            "access_charges": record["Access Charges"],
            "merchant_cancellation_charges": record["Merchant Cancellation Charges"],
            "call_centre_service_fee": record["Call Center Service Fees"],
            "swiggy_service_fee_without_tax": record["Total Swiggy Service fee (without taxes)"],
            "tax_on_swiggy_service_fee": record["Taxes on Swiggy fee"],
            "merchant_share_of_cancelled_orders": record["Merchant Share of Cancelled Orders"],
            "refund_on_disputed_orders": record["Refund for Disputed Order"],
            "disputed_orders_remarks": record["Disputed Order Remarks"],
            "order_level_adjustments": record["Total of Order Level Adjustments"],
            "net_payable_amount": record["Net Payable Amount (before TCS deduction)"],
            "net_payable_amount_after_tcs_tds": record["Net Payable Amount (after TCS and TDS deduction)"],
            "mfr_pressed": record["MFR Pressed?"],
            "cancellation_policy_applied": record["Cancellation Policy Applied"],
            "coupon_code_sourced": record["Coupon Code Sourced"],
            "discount_campaign_id": record["Discount Campaign ID"],
            "is_replicated": record["Is_replicated"],
            "base_order_id": record["Base order ID"],
            "mrp_items": record["MRP Items"],
            "order_payment_type": record["Order Payment Type"],
            "cancellation_time": record["Cancellation time"],
            "pick_up_status": record["Pick Up Status"],
            "coupon_code_applied_by_customer": record["Coupon code applied by customer"],
            "long_distance_applicable": record["Long Distance Applicable"],
            "last_mile_distance": record["Last Mile Distance"],
            "long_distance_subscription_fees": record["Total Long Distance Subscription Fees"],
            "discount_on_long_distance_subscription_fees": record["Total Discount on Long Distance Subscription Fees"],
            "net_long_distance_subscription_fees": record["Total Effective Long Distance Subscription Fees"]
        }
        
        data.append(dct)
    
    if len(data) > 0:
        query = """INSERT INTO swiggy_order_finance_metrics (
                order_id,
                restaurant_id,
                order_date,
                created_at,
                updated_at,
                order_status,
                order_category,
                cancelled_by,
                item_total,
                packing_and_service_charges,
                merchant_discount,
                net_amount_without_taxes,
                gst_liability_of_merchant,
                customer_payable_net_after_tax_and_discount,
                platform_fee_chargeable_on,
                platform_fee_percent,
                platform_fee,
                discount_on_platform_fee,
                collection_charges,
                access_charges,
                merchant_cancellation_charges,
                call_centre_service_fee,
                swiggy_service_fee_without_tax,
                tax_on_swiggy_service_fee,
                merchant_share_of_cancelled_orders,
                refund_on_disputed_orders,
                disputed_orders_remarks,
                order_level_adjustments,
                net_payable_amount,
                net_payable_amount_after_tcs_tds,
                mfr_pressed,
                cancellation_policy_applied,
                coupon_code_sourced,
                discount_campaign_id,
                is_replicated,
                base_order_id,
                mrp_items,
                order_payment_type,
                cancellation_time,
                pick_up_status,
                coupon_code_applied_by_customer,
                long_distance_applicable,
                last_mile_distance,
                long_distance_subscription_fees,
                discount_on_long_distance_subscription_fees,
                net_long_distance_subscription_fees
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s)
            ON CONFLICT (order_id,restaurant_id)
            DO NOTHING;
            """
        
        for record in data:
            # Clean and prepare data
            record = {key: str(value).replace('%', '') for key, value in record.items()}
            execute_query(query, tuple(record.values()))
        
        connection.commit()
    else:
        printLog("--No data to save.")


In [9]:
process_report(r"C:\Users\VM-jampacked\Downloads\finance_data - finance_data1aprto25aug.csv")

--Report downloaded. Extracting content...
--55960 records available.


In [ ]:
import pandas as pd
df = pd.read_csv

In [19]:
import requests
import time
import logging
import json

# Set up logging
logging.basicConfig(
    filename='api_debug.log', 
    level=logging.DEBUG,
    format='%(asctime)s %(levelname)s:%(message)s'
)

# Constants
base_url = 'https://rms.swiggy.com/orders/v1/history?'
filters_ = ['wrong_items', 'unsafe_packaging', 'missing_items', 'quality_issues', 'packaging_issues', 'quantity_issues']
access_token = 'f4458319-bfa5-4295-a9a1-d9e1be2b23b7'
start_date = '2024-08-20'
end_date = '2024-09-26'
restaurant_id = '765524'

def fetch_orders_by_filter(base_url, access_token, filter_, start_date, end_date, restaurant_id):
    params = {
        'filterReq': filter_,
        'limit': 100,
        'offset': 0,
        'ordered_time__gte': start_date,
        'ordered_time__lte': end_date,
        'restaurant_id': restaurant_id
    }
    
    headers = {
        'authority': 'rms.swiggy.com',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'accesstoken': access_token,
        'content-type': 'application/json',
        'origin': 'https://partner-self-client.swiggy.com',
        'referer': 'https://partner-self-client.swiggy.com/',
        'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    }
    
    while True:
        try:
            response = requests.get(base_url, headers=headers, params=params)
            if response.status_code == 200:
                response_data = response.json()

                # Process data
                res_data = response_data['data'][0]['data']['objects']
                return res_data
            else:
                logging.error(f'Error occurred: {response.status_code} - {response.text}')
                return []
        except Exception as e:
            logging.error(f'Exception occurred: {str(e)}')
            return []

def main():
    filter_lists = {filter_: [] for filter_ in filters_}

    for filter_ in filters_:
        logging.info(f'Fetching orders for filter: {filter_}')
        orders = fetch_orders_by_filter(base_url, access_token, filter_, start_date, end_date, restaurant_id)
        if orders:
            filter_lists[filter_].extend(orders)

    # Process the collected orders
    for filter_, orders in filter_lists.items():
        logging.info(f'Orders for {filter_}: {len(orders)}')
        # You can perform further processing on orders here

if __name__ == "__main__":
    main()
